In [8]:
feedback_produto = """"
O Galaxy a54 me surpreendeu positivamente em vários aspectos. O design é elegante e leve, com acabamento premium que passa uma ótima sensação ao segurar. A tela tem cores vibrantes e excelente definição, ideal para quem consome muitos vídeos ou jogos.

O desempenho é muito bom: aplicativos abrem rapidamente e não percebi travamentos, mesmo com várias tarefas simultâneas. A câmera entrega fotos de alta qualidade, especialmente em ambientes bem iluminados, e o modo noturno também é satisfatório.

A bateria tem uma boa duração, aguentando um dia inteiro de uso moderado a intenso, e o carregamento rápido realmente cumpre o que promete.

Como ponto de melhoria, achei que o aparelho poderia oferecer uma resistência maior à água e o preço poderia ser um pouco mais competitivo em relação a modelos similares.

No geral, estou muito satisfeito com a compra e considero o smartphone uma excelente escolha para quem busca desempenho e qualidade em um único aparelho.
"""

In [9]:
from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template(
    """
Dado o texto de avaliação de um cliente, extraia as seguintes informações e retorne um objeto JSON:

- produto: Nome do produto mencionado no texto.
- caracteristicas_positivas: Lista de todas as características positivas destacadas sobre o produto.
- caracteristicas_negativas: Lista de todas as características negativas apontadas sobre o produto.
- recomendacao: "sim" se o cliente recomenda o produto; caso contrário, "não".

Exemplo de saída JSON:
{{
  "produto": "Nome do Produto",
  "caracteristicas_positivas": ["excelente desempenho","bateria de longa duração"],
  "caracteristicas_negativas": ["preço elevado","falta de resistência à água"],
  "recomendacao": "sim"
}}

Texto da avaliação:
{review}
"""
)

# Para exibir apenas o conteúdo formatado (sem mostrar roles), por exemplo:
mensagens = review_template.format_messages(review=feedback_produto)
for msg in mensagens:
    print(msg.content)


Dado o texto de avaliação de um cliente, extraia as seguintes informações e retorne um objeto JSON:

- produto: Nome do produto mencionado no texto.
- caracteristicas_positivas: Lista de todas as características positivas destacadas sobre o produto.
- caracteristicas_negativas: Lista de todas as características negativas apontadas sobre o produto.
- recomendacao: "sim" se o cliente recomenda o produto; caso contrário, "não".

Exemplo de saída JSON:
{
  "produto": "Nome do Produto",
  "caracteristicas_positivas": ["excelente desempenho","bateria de longa duração"],
  "caracteristicas_negativas": ["preço elevado","falta de resistência à água"],
  "recomendacao": "sim"
}

Texto da avaliação:
"
O Galaxy a54 me surpreendeu positivamente em vários aspectos. O design é elegante e leve, com acabamento premium que passa uma ótima sensação ao segurar. A tela tem cores vibrantes e excelente definição, ideal para quem consome muitos vídeos ou jogos.

O desempenho é muito bom: aplicativos abrem ra

In [10]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.6)

response = chat.invoke(mensagens)

print(response)

content='{\n  "produto": "Galaxy a54",\n  "caracteristicas_positivas": ["design elegante e leve","tela com cores vibrantes e excelente definição","bom desempenho","câmera de alta qualidade","bateria com boa duração","carregamento rápido"],\n  "caracteristicas_negativas": ["falta de resistência à água","preço elevado"],\n  "recomendacao": "sim"\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 442, 'total_tokens': 545, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BfX0cLaJHF0ToPbBDbPyeogmwlmcd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--771188b3-5483-4530-b5ab-95b616a9f0fa-0' usage_metadata={'input_tokens': 442, 'output_tokens': 103, 'total_tokens'

In [11]:
from langchain.output_parsers import ResponseSchema

from langchain.output_parsers import StructuredOutputParser

produto_schema = ResponseSchema(
    name="produto", description="Nome do produto mencionado no texto.", type="str"
)

caracteristicas_ativas_schema = ResponseSchema(
    name="caracteristicas_positivas",
    description="Lista de todas as características positivas destacadas sobre o produto.",
    type="list"
)

caracteristicas_negativas_schema = ResponseSchema(
    name="caracteristicas_negativas",
    description="Lista de todas as características negativas apontadas sobre o produto.",
    type="list",
)

recomendacao_schema = ResponseSchema(
    name="recomendacao",
    description="“sim” se o cliente recomenda o produto; caso contrário, “não”.",
    type="bool",
)

# 2. Monte a lista de schemas usando as variáveis criadas acima
response_schema = [
    produto_schema,
    caracteristicas_ativas_schema,
    caracteristicas_negativas_schema,
    recomendacao_schema,
]

print(response_schema)

[ResponseSchema(name='produto', description='Nome do produto mencionado no texto.', type='str'), ResponseSchema(name='caracteristicas_positivas', description='Lista de todas as características positivas destacadas sobre o produto.', type='list'), ResponseSchema(name='caracteristicas_negativas', description='Lista de todas as características negativas apontadas sobre o produto.', type='list'), ResponseSchema(name='recomendacao', description='“sim” se o cliente recomenda o produto; caso contrário, “não”.', type='bool')]


In [12]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

formated_schema = output_parser.get_format_instructions()

print(formated_schema)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"produto": str  // Nome do produto mencionado no texto.
	"caracteristicas_positivas": list  // Lista de todas as características positivas destacadas sobre o produto.
	"caracteristicas_negativas": list  // Lista de todas as características negativas apontadas sobre o produto.
	"recomendacao": bool  // “sim” se o cliente recomenda o produto; caso contrário, “não”.
}
```


In [13]:
review_template2 = ChatPromptTemplate.from_template("""
Para o texto a seguir extraia as informações baseado no schema;         

Texto: {review}

Schema: {schema}                                           
""", partial_variables={"schema": formated_schema, 'review': feedback_produto})

In [14]:
from typing import TypedDict, List

class ReviewOutput(TypedDict):
    produto: str
    caracteristicas_positivas: List[str]
    caracteristicas_negativas: List[str]
    recomendacao: str


response = chat.invoke(review_template2.format_messages())

response.content

'```json\n{\n\t"produto": "Galaxy a54",\n\t"caracteristicas_positivas": ["Design elegante e leve", "Tela com cores vibrantes e excelente definição", "Desempenho muito bom", "Câmera entrega fotos de alta qualidade", "Bateria com boa duração", "Carregamento rápido"],\n\t"caracteristicas_negativas": ["Falta de resistência maior à água", "Preço poderia ser mais competitivo"],\n\t"recomendacao": "sim"\n}\n```'

In [15]:
response_json: ReviewOutput = output_parser.parse(response.content)


response_json.get('caracteristicas_negativas')

['Falta de resistência maior à água', 'Preço poderia ser mais competitivo']